<a href="https://colab.research.google.com/github/minhducvo04/Math_Monsters/blob/main/UI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai
!pip install gradio
!pip googledrivedownloader

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 66.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 12.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 102.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 10.6 

In [ ]:
import requests
import numpy as np
# prompt: read an image from a file and convert it so that the openai gpt can read it
import openai
from PIL import Image
import io
from io import BytesIO
import base64
from openai import OpenAI
import gradio as gr
from google.colab import userdata
import json
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
default_headers = {
    'app_id': userdata.get('APP_ID'),
    'app_key': userdata.get('APP_KEY'),
    'Content-type': 'application/json'
}

service = 'https://api.mathpix.com/v3/latex'


assistant_message = r"""\documentclass{article}
\usepackage{amsmath}

\begin{document}

\section*{Evaluating the Definite Integral}

The given problem is to evaluate the definite integral \(\int_{0}^{3} x^{3} \, dx\). Let's go through the correct steps to solve this problem.

\section*{1. Correct Antiderivative}

To find the antiderivative of \( x^3 \):

\[
\int x^3 \, dx = \frac{x^4}{4} + C
\]

\section*{2. Evaluate the Definite Integral}

Next, we evaluate the definite integral from 0 to 3:

\[
\int_{0}^{3} x^3 \, dx = \left. \frac{x^4}{4} \right|_{0}^{3}
\]

\section*{3. Substitute the Limits}

Substitute the upper and lower limits into the antiderivative:

\[
\left. \frac{x^4}{4} \right|_{0}^{3} = \frac{3^4}{4} - \frac{0^4}{4}
\]

\section*{4. Calculate the Values}

Calculate the values of the expression:

\[
\frac{3^4}{4} = \frac{81}{4}, \quad \frac{0^4}{4} = 0
\]

Therefore, the value of the definite integral is:

\[
\frac{81}{4} - 0 = \frac{81}{4}
\]

\end{document}"""

assistant_message2 = r"""**Problem:**

Evaluate the integral:

\[
\int_{0}^{1} x^2 \, dx
\]

**Proposed Solution:**

\[
\int_{0}^{1} x^2 \, dx = \frac{1}{3}
\]

**Evaluation:**

Let's evaluate the integral step-by-step to verify the solution.

First, we find the antiderivative of \(x^2\):

\[
\int x^2 \, dx
\]

The antiderivative is found by using the power rule of integration:

\[
\int x^n \, dx = \frac{x^{n+1}}{n+1} + C
\]

For \(x^2\), we have:

\[
\int x^2 \, dx = \frac{x^{2+1}}{2+1} = \frac{x^3}{3}
\]

Next, we evaluate this antiderivative from 0 to 1:

\[
left[ \frac{x^3}{3} right]_{0}^{1}
\]

Calculating the values:

\[
\left. \frac{x^3}{3} \right|_{0}^{1} = \frac{1^3}{3} - \frac{0^3}{3} = \frac{1}{3} - 0 = \frac{1}{3}
\]

The proposed solution correctly follows the integration steps and correctly evaluates the definite integral.

**Final Answer:**

Correct

\[
\int_{0}^{1} x^2 \, dx = \frac{1}{3}
\]

"""

#
# Return the base64 encoding of an image with the given filename.
#
def image_uri(filename):
    image_data = open(filename, "rb").read()
    return "data:image/jpg;base64," + base64.b64encode(image_data).decode()

#
# Call the Mathpix service with the given arguments, headers, and timeout.
#
def latex(args, headers=default_headers, timeout=30):
    r = requests.post(service,
        data=json.dumps(args), headers=headers, timeout=timeout)
    return json.loads(r.text)

In [ ]:
memory = []

def numpy_to_base64(img_array):
    # Convert NumPy array to PIL Image
    img = Image.fromarray(img_array.astype('uint8'), 'RGB')

    # Create a buffer to hold the image data
    buffered = BytesIO()

    # Save the image to the buffer in PNG format
    img.save(buffered, format="PNG")

    # Get the byte data from the buffer
    img_bytes = buffered.getvalue()

    # Encode the byte data to base64
    img_base64 = base64.b64encode(img_bytes).decode('utf-8')

    return img_base64

def format_latex(text):
    # Replace inline LaTeX placeholders with actual LaTeX delimiters
    text = text.replace('[', '$').replace(']', '$')
    # Replace block LaTeX placeholders with actual LaTeX delimiters
    text = text.replace('\\[', '$$').replace('\\]', '$$')
    return text

def read_image(input_img):
    img = numpy_to_base64(input_img)

    to_process = latex(args = {
        'src': f"data:image/png;base64,{img}",
        'ocr': ['math', 'text'],
        'skip_recrop': True,
        'formats': ['text', 'latex_styled', 'asciimath', 'mathml'],
        'format_options': {
            'text': {
                'transforms': ['rm_spaces', 'rm_newlines'],
                'math_delims': ['$', '$']
            },
            'latex_styled': {'transforms': ['rm_spaces']}
        }
    })['text']

    # b64_image = numpy_to_base64(input_img)
    model = OpenAI(api_key = userdata.get('OPENAI_API_KEY'))
    response = model.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system",
         "content": [
            {
                "type": "text",
                "text": f"""You are a mathematics and physics problem evaluator. You will provide concise and accurate answers. You will receive a math problem as input along with a proposed solution. Your task is to determine if the final answer is correct, but do not display any information until you finish calculating. If the answer is correct, end your response with 'Correct' after you finish processing and calculating, followed by an explanation in Markdown format. You will want to explain and elaborate more on the intermediary steps if the working provided is not satisfactory. If the answer is incorrect, end your response with 'Incorrect' followed by a detailed explanation in Markdown format on what is wrong with the solution and how it should be solved correctly. Always format your responses in Markdown, ensuring to use appropriate LaTeX formatting for mathematical expressions. Be accurate with the LaTeX. Make sure that the delimiters are correct. Here is an example of perfect LaTeX: {assistant_message}"""
                }]
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Here is the problem: \[\int_{0}^{1} x^2 \, dx = \frac{1}{3}\]. Can you identify if my working is correct? if it is not, can you tell me what went wrong and explain step by step and what should have been done to reach the correct answer? You may use LaTeX format. "
                }
            ]
        },
        {"role": "assistant",
         "content": [
            {
                "type": "text",
                "text": assistant_message2
            }
         ]
        },
        {"role": "user",
        "content": [
            # {
            #     "type": "image_url",
            #     "image_url": {"url": f"data:image/jpeg;base64,{img}"}
            # },
            {
                "type": "text",
                "text": f"Here is the problem: {to_process}. Can you identify if my working is correct? if it is not, can you tell me what went wrong and explain step by step and what should have been done to reach the correct answer? You may use LaTeX format. ",
            }]
        }
    ],
    max_tokens=1024,
    temperature=0.9,
    )
    # print(response)
    message = response.choices[0].message
    message_text = message.content
    print(message_text)



    if message_text.endswith("correct"):
      print("uhuh")

    # Format the LaTeX in the message_text
    formatted_message_text = format_latex(message_text)

    memory.append(message_text)
    return to_process, formatted_message_text

demo = gr.Interface(fn=read_image, inputs=["image"], outputs=[gr.Markdown(latex_delimiters = [{"left": "$", "right": "$"}]), gr.Markdown(latex_delimiters = [{"left": "$", "right": "$"}])])
demo.launch(share=True, debug = True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://cead57426e80743f6e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


**Problem:**

Evaluate the integral:

\[
\int_{1}^{2} 2x \, dx
\]

**Proposed Solution:**

\[
\int_{1}^{2} 2x \, dx = \left[x^{2}\right]_{1}^{2} = 2^{2} - 1^{2} = 3
\]

**Evaluation:**

Let's evaluate the integral step-by-step to verify the solution. 

First, we need to find the antiderivative of \(2x\):

\[
\int 2x \, dx
\]

Using the power rule of integration:

\[
\int x^n \, dx = \frac{x^{n+1}}{n+1} + C
\]

For \(2x\), we have:

\[
\int 2x \, dx = 2 \int x \, dx = 2 \left(\frac{x^2}{2}\right) = x^2
\]

Next, we evaluate this antiderivative from 1 to 2:

\[
\left. x^2 \right|_{1}^{2}
\]

Calculating the values:

\[
\left. x^2 \right|_{1}^{2} = 2^2 - 1^2 = 4 - 1 = 3
\]

The proposed solution correctly follows the integration steps and correctly evaluates the definite integral.

**Final Answer:**

Correct

\[
\int_{1}^{2} 2x \, dx = 3
\]

However, note that the proposed solution has an error in formatting. Correct solution should show the intermediate antiderivative step explicitly as 